In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Check file existence
filename = 'Libro1.xlsx'
# Read data from Excel
datosDF = pd.read_excel(filename)
df = pd.DataFrame(datosDF)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/camilogonzalez/Library/Python/3.11/lib/python/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/camilogonzalez/Library/Python/3.11/lib/python/site-packages/traitlets/config/application.py", line 1053, in launch_instance
    app.start()
  File "/Users/camilogonzalez/Library/Python/3.11/lib/python/site-packages/ipykernel/kernelapp.py", line 737, in start

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/camilogonzalez/Library/Python/3.11/lib/python/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/camilogonzalez/Library/Python/3.11/lib/python/site-packages/traitlets/config/application.py", line 1053, in launch_instance
    app.start()
  File "/Users/camilogonzalez/Library/Python/3.11/lib/python/site-packages/ipykernel/kernelapp.py", line 737, in start

AttributeError: _ARRAY_API not found

In [ ]:
ss = StandardScaler()
scaler = ss.fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [ ]:
numbers_of_rows = datosDF.values.shape[0]
window_length = 14
number_of_features = 14  # Ahora solo 14 números por combinación
train = np.empty([numbers_of_rows-window_length, window_length, number_of_features], dtype=float)
label = np.empty([numbers_of_rows-window_length, number_of_features], dtype=float)

for i in range(0, numbers_of_rows-window_length):
    train[i] = transformed_df.iloc[i:i+window_length, 0:number_of_features]
    label[i] = transformed_df.iloc[i+window_length, 0:number_of_features]

In [ ]:
def exact_match_accuracy(y_true, y_pred):
    # Redondear y convertir a enteros
    y_pred_rounded = np.round(y_pred).astype(int)
    y_true_int = np.round(y_true).astype(int)
    # Comparar coincidencia exacta de los 14 números
    matches = np.all(y_pred_rounded == y_true_int, axis=1)
    return np.mean(matches)

# Separar datos en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(train, label, test_size=0.2, random_state=42)

# Callbacks para entrenamiento robusto
early_stop = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, min_lr=1e-7)

model = Sequential()
model.add(Bidirectional(LSTM(512, input_shape=(window_length, number_of_features), return_sequences=True)))
model.add(Dropout(0.4))
model.add(Bidirectional(LSTM(512, return_sequences=True)))
model.add(Dropout(0.4))
model.add(Bidirectional(LSTM(512, return_sequences=True)))
model.add(Bidirectional(LSTM(512, return_sequences=False)))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dense(number_of_features))

model.compile(loss='mse', optimizer=Adam(learning_rate=0.00001))

# Entrenamiento intensivo
model.fit(X_train, y_train, epochs=500, batch_size=8, verbose=1, validation_data=(X_val, y_val), callbacks=[early_stop, lr_reduce])

# Evaluación personalizada y reporte de errores
val_pred = model.predict(X_val)
val_acc = exact_match_accuracy(y_val, val_pred)
print(f'Exact match accuracy en validación: {val_acc*100:.2f}%')

# Mostrar ejemplos donde falla la predicción
for i in range(len(y_val)):
    pred = np.round(scaler.inverse_transform([val_pred[i]])).astype(int)[0]
    real = np.round(scaler.inverse_transform([y_val[i]])).astype(int)[0]
    if not np.array_equal(pred, real):
        print(f'Fallo en muestra {i}: Real={real}, Predicho={pred}')

# Predicción del siguiente sorteo
last_rows = df.tail(window_length)
to_predict = np.array(last_rows)
scales_to_predict = scaler.transform(to_predict)
scales_to_predict = np.expand_dims(scales_to_predict, axis=0)
predicted_output = model.predict(scales_to_predict)
predicted_output = np.round(scaler.inverse_transform(predicted_output)).astype(int)[0]
print('Predicción del siguiente sorteo (14 números):', predicted_output)
